In [1]:
import glob
import pandas as pd
import re
import nilearn as nil
from nilearn import *
from nilearn import image
from os.path import basename
import numpy as np
import yaml
from yaml.loader import SafeLoader
from socket import gethostname

from get_all_series import get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed
from get_all_series import get_all_subj_df

print(gethostname(),flush=True)
# Open the file and load the file
with open('config.yml') as f:
    all_yaml = yaml.load(f, Loader=SafeLoader)
    if gethostname() in all_yaml.keys():
        config = all_yaml[gethostname()]
    else:
        config = all_yaml['default']
        
print(config,flush=True)

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'fmriprep_dir': '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep_2022/', 'nii_raw_path': 'sub-DEV*/ses-wave*/func/s6_sub-DEV*_ses-wave*_task-SST_acq-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii', 'mask_location': '/

In [2]:
dropbox_data_dir = config['dropbox_data_dir']

In [5]:
import pickle

#load roi_data_raw.pkl
roi_data_raw = pickle.load(open(dropbox_data_dir + 'roi_data_raw.pkl','rb'))

In [10]:
#get the number of wave 1 and 2 values
", ".join([str(s.keys()) for s in roi_data_raw.values()])

"dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1']), dict_keys(['1']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1', '2']), dict_keys(['1']), dict_keys(['1', '2']), dict_keys(['

So it seems like we do in fact have 2 run of ROI data for most subjects.

In [11]:
sst_all_behavioral_data = pd.read_csv(dropbox_data_dir + "/sst_behavioral_data_all.csv")

In [12]:
sst_all_behavioral_data

,trial_n,numchunks_reciprocal,go_no_go_condition,arrow_presented,ladder_number,LadderX_SSD_ms,subject_response,ladder_movement,reaction_time,SSD_technical,...,subid,waveid,runid,rundatetime,go_no_go_condition_label,condition,health_condition_label,jitter_value,OCI,onset_arrow
0,1,1,0,1,0,0,94,0,0.630607,0,...,DEV004,1,1,2018-04-17T02:18:00Z,Go,CorrectGo,Healthy,0.75834,1.25834,1.305157
1,2,1,2,2,0,0,0,0,0.000000,0,...,DEV004,1,1,2018-04-17T02:18:00Z,Cue,Cue,NaN,NaN,NaN,NaN
2,3,1,1,0,1,250,91,-1,0.467996,0,...,DEV004,1,1,2018-04-17T02:18:00Z,Stop,FailedStop,Unhealthy,0.50556,1.00556,4.022672
3,4,1,2,2,0,0,0,0,0.000000,0,...,DEV004,1,1,2018-04-17T02:18:00Z,Cue,Cue,NaN,NaN,NaN,NaN
4,5,1,0,0,0,0,91,0,0.432759,0,...,DEV004,1,1,2018-04-17T02:18:00Z,Go,CorrectGo,Healthy,0.75834,1.25834,7.662265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285691,252,3,2,2,0,0,0,0,0.000000,0,...,DEV999,3,1,2019-12-11T20:25:00Z,Cue,Cue,NaN,NaN,NaN,NaN
285692,253,3,0,1,0,0,21,0,0.524493,0,...,DEV999,3,1,2019-12-11T20:25:00Z,Go,CorrectGo,Healthy,1.01110,1.51110,421.097104
285693,254,3,2,2,0,0,0,0,0.000000,0,...,DEV999,3,1,2019-12-11T20:25:00Z,Cue,Cue,NaN,NaN,NaN,NaN
285694,255,3,0,1,0,0,21,0,0.430193,0,...,DEV999,3,1,2019-12-11T20:25:00Z,Go,CorrectGo,Unhealthy,0.00000,0.50000,423.101896
